

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/Language_Detector.ipynb)




# **Identify Language of a text**

## 1. Colab Setup

In [1]:
# Install java
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
!java -version

# Install pyspark
!pip install --ignore-installed -q pyspark==2.4.4

# Install Sparknlp
!pip install --ignore-installed spark-nlp

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
     |████████████████████████████████| 215.7MB 65kB/s 
     |████████████████████████████████| 204kB 56.0MB/s 
     |████████████████████████████████| 133kB 7.7MB/s 


In [2]:
import pandas as pd
import numpy as np
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Setup Spark Session

In [3]:
spark = sparknlp.start()

## 3. Select the DL model and re-run cells below

In [4]:
model_name = 'ld_wiki_7'
#model_name = 'ld_wiki_20'

## 4. Some sample examples

In [5]:
## Generating Example Files ##
if model_name == 'ld_wiki_7':
  text_list = [#English
             "William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist. He is best known as the co-founder of Microsoft Corporation. During his career at Microsoft, Gates held the positions of chairman, chief executive officer (CEO), president and chief software architect, while also being the largest individual shareholder until May 2014.",
#German
"""Als Sebastian Thrun 2007 bei Google anfing, an selbstfahrenden Autos zu arbeiten, nahmen ihn nur wenige Leute außerhalb des Unternehmens ernst.""",
#Spanish
 """La historia del procesamiento del lenguaje natural generalmente comenzó en la década de 1950, aunque se puede encontrar trabajo de períodos anteriores. En 1950, Alan Turing publicó un artículo titulado 'Maquinaria de computación e inteligencia' que proponía lo que ahora se llama la prueba de Turing como criterio de inteligencia""",
#Italian
 """Geoffrey Everest Hinton è uno psicologo cognitivo e uno scienziato informatico canadese inglese, noto soprattutto per il suo lavoro sulle reti neurali artificiali. Dal 2013 divide il suo tempo lavorando per Google e l'Università di Toronto. Nel 2017 è stato cofondatore ed è diventato Chief Scientific Advisor del Vector Institute di Toronto.""",
#czech
 """Steven Paul Jobs byl americký obchodní magnát, průmyslový designér, investor a vlastník médií.""",
#slovak
 """Titanic je americký epický román a katastrofický film z roku 1997, ktorý režíroval, napísal, spoluprodukoval a spolu-editoval James Cameron. Zahŕňa historické aj beletrizované aspekty a je založený na správach o potopení RMS Titanicu a hviezd Leonarda DiCaprio a Kate Winsletovej ako členov rôznych sociálnych vrstiev, ktorí sa na palube lode zaľúbia počas svojej nešťastnej prvej plavby.""",
#German
 "Als ich John sagte, dass ich nach Alaska ziehen wollte, warnte er mich, dass ich dort Probleme haben würde, einen Starbucks zu finden.",
#English
"Other than being the king of the north, John Snow is a an english physician and a leader in the development of anaesthesia and medical hygiene. He is considered for being the first one using data to cure cholera outbreak in 1834.",
#French
 """Facebook est un service de réseautage social lancé sous le nom de TheFacebook le 4 février 2004. Il a été fondé par Mark Zuckerberg avec ses colocataires et ses collègues étudiants de l'Université Harvard Eduardo Saverin, Andrew McCollum, Dustin Moskovitz et Chris Hughes.""",
 #French
"La Joconde est une peinture à l'huile du XVIe siècle créée par Léonard. Il se tient au Louvre à Paris.",
]
else:
  text_list=[
             #English
"""Mona Lisa is a 16th century oil painting created by Leonardo. It is held at the Louvre in Paris.""",
           #French
"""La Joconde est une peinture à l'huile du XVIe siècle créée par Léonard. Il se tient au Louvre à Paris.""",
           #Bulgarian
"""Джефри Еверест Хинтън е британски канадски когнитивен психолог и компютърен учен, най-известен с работата си върху изкуствени невронни мрежи. От 2013 г. той прекарва времето си в работа за Google и университета в Торонто. През 2017 г. е съосновател и става главен научен съветник на Vector Institute of Toronto.""",
#Czech
"""Mona Lisa je olejomalba ze 16. století, kterou vytvořil Leonardo. Koná se v Louvru v Paříži.""",
#German
"""Titanic ist ein 1997 in den USA veröffentlichter epischer Roman und ein katastrophaler Film, der von James Cameron inszeniert, geschrieben, co-produziert und mitherausgegeben wurde. Es deckt sowohl historische als auch fiktive Aspekte ab und basiert auf Berichten über den Untergang der RMS Titanic und der Stars Leonard DiCaprio und Kate Winslet als Mitglieder verschiedener sozialer Klassen, die sich während der Schiffsreise während ihrer unglücklichen ersten Reise verlieben.""",
#Greek
"""Το Titanic είναι ένα αμερικανικό επικό μυθιστόρημα του 1997 και μια καταστροφική ταινία σε σκηνοθεσία, συγγραφή, συμπαραγωγή και συν-επεξεργασία από τον James Cameron. Καλύπτει τόσο ιστορικές όσο και φανταστικές πτυχές και βασίζεται σε αναφορές για τη βύθιση του Τιτανικού RMS και πρωταγωνιστούν οι Leonard DiCaprio και Kate Winslet ως μέλη διαφόρων κοινωνικών τάξεων που ερωτεύονται κατά τη διάρκεια του ταξιδιού του πλοίου κατά τη διάρκεια του πρώτου τους ατυχούς ταξιδιού.""",
#Spanish
"""Geoffrey Everest Hinton es un psicólogo cognitivo y científico informático británico canadiense, mejor conocido por su trabajo en redes neuronales artificiales. Desde 2013 ha pasado su tiempo trabajando para Google y la Universidad de Toronto. En 2017 fue cofundador y se convirtió en Asesor Científico Jefe del Instituto Vector de Toronto.""",
#Finnish
"""Mona Lisa on 1500-luvun öljymaalaus, jonka on luonut Leonardo. Se pidetään Pariisin Louvressa.""",
#Croatian
"""Povijest obrade prirodnog jezika općenito je počela 1950-ih, iako se mogu naći djela iz ranijih razdoblja. 1950. Alan Turing objavio je članak pod naslovom "Računalna i inteligencijska mašinerija" u kojem se kao kriterij inteligencije predlaže ono što se danas naziva Turingov test.""",
#Hungarian
"""A természetes nyelvfeldolgozás története általában az 1950-es években kezdődött, bár a korábbi időszakokból származó munkák is megtalálhatók. 1950-ben Alan Turing közzétett egy cikket, melynek címe: „Számítástechnika és intelligenciagépek”, és amely intelligenciakritériumként javasolta a Turing-tesztet.""",
#Norwegian
"""Titanic er en amerikansk episk roman fra 1997 og katastrofal film regissert, skrevet, co-produsert og co-redigert av James Cameron. Det dekker både historiske og fiksjoniserte aspekter og er basert på rapporter om synkingen av RMS Titanic og stjernene Leonard DiCaprio og Kate Winslet som medlemmer av forskjellige sosiale klasser som forelsker seg under skipets seilas under deres uheldige første seilas.""",
#Polish
"""Poza tym, że jest królem północy, John Snow jest angielskim lekarzem i liderem w dziedzinie anestezjologii i higieny medycznej. Uważany jest za pierwszego, który wykorzystał dane do leczenia epidemii cholery w 1834 r.""",
#Portuguese
"""Geoffrey Everest Hinton é um psicólogo cognitivo britânico canadense e cientista da computação, mais conhecido por seu trabalho em redes neurais artificiais. Desde 2013, ele trabalha para o Google e a Universidade de Toronto. Em 2017, foi co-fundador e tornou-se Conselheiro Científico Chefe do Vector Institute of Toronto.""",
#Romanian
"""Mona Lisa este o pictură în ulei din secolul al XVI-lea creată de Leonardo. Se ține la Louvre din Paris.""",
#Russian
"""Когда в 2007 году Себастьян Трун начал работать над машинами для самостоятельного вождения в Google, мало кто за пределами компании воспринимал его всерьез.""",
#Slovak
"""John Snow je okrem anglického kráľa anglickým lekárom a lídrom vo vývoji anestézie a lekárskej hygieny. Je považovaný za prvého, ktorý používa údaje na liečenie prepuknutia cholery v roku 1834.""",
#Swedish
"""Mona Lisa är en oljemålning från 1500-talet skapad av Leonardo. Det hålls vid Louvre i Paris.""",
#Turkish
"""Mona Lisa, Leonardo tarafından yaratılan 16. yüzyıldan kalma bir yağlı boyadır. Paris'teki Louvre'da düzenleniyor.""",
#Ukrainian
"""Facebook - це послуга соціальних мереж, запущена під назвою TheFacebook 4 лютого 2004 року. Його заснував Марк Цукерберг разом зі своїми одноквартами та колегами Гарвардського університету Едуардо Саверином, Ендрю Макколлумом, Дастіном Московіцем та Крісом Х'юзом.""",
#Italian
"""Quando Sebastian Thrun ha iniziato a lavorare su auto a guida autonoma presso Google nel 2007, poche persone al di fuori dell'azienda lo hanno preso sul serio.""",
]


## 5. Define Spark NLP pipeline

In [6]:
documentAssembler = DocumentAssembler()\
                    .setInputCol("text")\
                    .setOutputCol("document")

languageDetector = LanguageDetectorDL.pretrained(model_name)\
      .setInputCols("document")\
      .setOutputCol("language")\
      .setThreshold(0.5)\
      .setCoalesceSentences(True)

nlpPipeline = Pipeline(stages=[ documentAssembler, 
                                 languageDetector
                                 ])




ld_wiki_7 download started this may take some time.
Approximate size to download 3 MB
[OK!]


## 6. Run the pipeline

In [7]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(df)

## 7. Visualize results

In [8]:
result.select(F.explode(F.arrays_zip('document.result', 'language.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("Document"),
        F.expr("cols['1']").alias("Language")).show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Document                                                                                                                                                                                                                                                                                                                                                                                                        |Language|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------